# Spatial and Temporal Correlation Analysis

This tutorial demonstrates: 

1. how to extract time series of one grid cell and compare with point measurement

2. how to map the statistical relationship between two spatio-temporal variables. 

It also covers the following technical concepts:

* subsetting netCDF data from OPeNDAP
* grid extraction
* linear and rank correlation and their statistical significance.
* grid resampling (temporal, spatial)

In [ ]:
# As usual, we start with our imports
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
seaborn.set_style('dark')

### 1. validation of soil moisture estimation

In this task, we will evaluate the surface soil moisture estimation from S-GRAFS (Satellite-Guided Root-zone Analysis and Forecasting System) with in-situ soil moisture measurements from OzFlux network.

The S-GRAFS used a 'data-heavy, model-lite' approach to estimate surface soil wetness and root-zone soil wetness. The system was developed by Siyuan Tian and Luigi Renzullo from ANU-WALD to provide global near real time soil moisture. The near real-time passive microwave retrievals of top-layer (0-5 cm) soil moisture from the Soil Moisture Active/Passive (SMAP) were assimilated into a satellite rainfall (GPM, Global Precipitation Mission) driven soil moisture modelling system. 

OzFlux is a national ecosystem research network set up to provide the Australian and global ecosystem modelling communities with nationally consistent observations of energy, carbon and water exchange between the atmosphere and key Australian ecosystems.

These data are on the National Computational Infrastructure (NCI) and available through THREDDS:

S-GRAFS: 
http://dapds00.nci.org.au/thredds/catalog/ub8/au/S-GRAFS/catalog.html

OzFlux:
http://dap.ozflux.org.au/thredds/catalog.html


<img src="./data/OzFlux_sites.png" alt="drawing" width="1000" align="left"/>

### 1.1 loading and subsetting model data

We will look at daily soil moisture at 10km resolution from S-GRAFS for a region in south western New South Wales [-34.5, -36],[145.5, 147.000]

In [ ]:
grafs_url = 'http://dapds00.nci.org.au/thredds/dodsC/ub8/au/S-GRAFS/Surface_Wetness_from_API_analysis_window_Australia_2016.nc'

In [ ]:
ds = xr.open_dataset(grafs_url)
ds

#### Can you select all the data in the bounding box [-34.5, -36],[145.5, 147.000] ?

In [ ]:
# select all the data in the bounding box
lat_bounds = slice(-34.5, -36)
lon_bounds = slice(145.5, 147.000)

grafs_s0 = 

In [ ]:
# plot the soil wetness for selected days
grafs_s0.wetness.sel(time=slice('2016-06-01','2016-06-04')).plot.imshow(col='time',col_wrap=4, \
                                                                        cmap='gist_earth_r',robust=True)

### 1.2 extracting time series 

There is one soil moinitoring site from OzFlux network in this region called 'Yanco'. The GPS coordinates is -34.9893, 146.2907. 

We will extract the time series of soil wetness for the closest grid cell and compare with in-situ measurements

####  Can you find the nearest model grid to the in-situ site and plot the time-series of the model-simulated soil wetness?

In [1]:
# find the nearest grid in model 

# plot the time-series of model-simulated soil wetness


### 1.3 loading and processing the in-situ soil moisture measurements

The gound measurements of soil moisture is collected every 30mins. We will need to resample the data to daily average and compare with model simulation. Note that in-situ data always contain gaps or suspicious values. Data filtering is required. 

The data variable for surface soil moisture in the netCDF is called 'Sws'. 

In [ ]:
yanco_insitu = 'http://dap.ozflux.org.au/thredds/dodsC/ozflux/sites/Yanco/L3/default/Yanco_L3.nc'
yanco = xr.open_dataset(yanco_insitu)

In [ ]:
# check the temporal resolution and coverage
yanco.time

#### Can you plot the in-situ data for 2016 only?

In [ ]:
# plot the data for 2016 only


In [ ]:
# the missing values were saved as -9999 
yanco.sel(time=slice('2016-01-01','2016-12-31')).Sws.min()

In [ ]:
# select soil moisture for 2016 only and asign any negative soil moisture values to NaN
yanco_SM = yanco.sel(time=slice('2016-01-01','2016-12-31')).Sws
yanco_SM.data[yanco_SM.data<0] = np.nan
yanco_SM.plot()

In [ ]:
# resample the 0.5 hourly data to daily average data
yanco_daily = yanco_SM.resample(time='1D').mean().squeeze()
yanco_daily.plot()

#### Can you  resample the data to monthly and every 8-day?

#### Let's plot the two time-series

In [ ]:
# compare the model simulated soil moisture with in-situ measurments
# plot them in two yaxis since they have different units. 
fig, ax = plt.subplots(figsize=(10,3))
yanco_daily.plot() # in-situ
grafs_yanco.plot(color='red') # model 

### 1.4  correlation analysis

we will use the `pearsonr` and `spearmanr` functions from SciPy
to calculate the correlation between model soil moisture and in-situ soil moisture.
The linear (or Pearson, or parametric) correlation coefficient is the most 
commonly measure the strength of the relationship between two variables. It 
is particularly well suited if both variables are close to normally distributed 
and a linear relationship can be assumed. If the relationship seems non-linear,
then it would be better to calculate the rank(or Spearman, or non-parametric) correlation coefficient.

In any event, it is always a good idea to always check whether the rank 
correlation is very different from the linear correlation coefficient. If the 
two approaches produce _R_- and _p_-values, that lead to similar conclusions, 
then that strengthens your analysis. 

In [ ]:
# import modules
from scipy.stats import linregress, pearsonr, spearmanr

In [ ]:
linregress(yanco_daily, grafs_yanco)

#### Can you find the indices where values in both data sets are not NaN?

In [ ]:
# NaNs have to be removed 

In [ ]:
linregress(yanco_daily[notnan], grafs_yanco[notnan])

In [ ]:
spearmanr(yanco_daily[notnan], grafs_yanco[notnan])

In [ ]:
pearsonr(yanco_daily[notnan], grafs_yanco[notnan])

## 2 statistical relationship between two spatio-temporal variables

It is well kown that fuel moisture content (FMC) is an important fuel property for assessing wildfire hazard, since it influences fuel flammability and fire behavior.

What about surface soil moisture? If surface is wet, the fire risk should be low. There should be a strong negative correlation. Let's have a look at the soil moisture conditions and flammability over the grassland areas in this region

The flammability data were developed by Marta Yebra using MODIS data and the data are available here: http://dapds00.nci.org.au/thredds/catalog/ub8/au/FMC/c6/mosaics/catalog.html

The flammability data is updated very 8-day. To simply the case, we will resample the data to monthly data.

### 2.1 comparing data with different time and spatial resolution

In [ ]:
flam_url = 'http://dapds00.nci.org.au/thredds/dodsC/ub8/au/FMC/c6/mosaics/flam_c6_2016.nc'
flammability = xr.open_dataset(flam_url).sel(latitude=lat_bounds, longitude=lon_bounds)
flam_monthly = flammability.resample(time='1M').mean()

In [ ]:
# resample the soil moisture data to monthly soil moisture
grafs_s0_monthly = grafs_s0.resample(time='1M').mean()

In [ ]:
grafs_s0_monthly.wetness.plot.imshow(col='time',col_wrap=6, cmap='gist_earth_r',robust=True)

In [ ]:
flam_monthly.flammability.plot.imshow(col='time',col_wrap=6, cmap='OrRd',robust=True)

### 2.2 resample the data to a same spatial resolution

The flammability data is ~0.005deg resolution, whereas the soil moisture is 0.1deg. We will use the funtion `griddata` from scipy.interpolate to resample flammability data to 10km to be consistent with soil moisture data

In [ ]:
from scipy.interpolate import griddata

In [ ]:
help(griddata)

In [ ]:
# use np.meshgrid to make N-D coordinate arrays for vectorized evaluations of N-D scalar/vector fields over N-D grids
lon,lat = np.meshgrid(flam_monthly.longitude,flam_monthly.latitude.data) #original resolution
lonnew,latnew = np.meshgrid(grafs_s0.lon,grafs_s0.lat) #target resolution
lonnew.shape

In [ ]:
data = flam_monthly.isel(time=0).flammability.data
output = griddata((lat.ravel(),lon.ravel()),\
                      data.ravel(),(latnew,lonnew), method='linear',fill_value=np.nan)

In [ ]:
# checked the output of the resampled data
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(data)
plt.subplot(122)
plt.imshow(output)

#### Now, we will resample all the monthly fammability data to 10km to calculate the temporal correlation for each pixel

In [ ]:
# create a dataset for the resampled flammability 
flam_10km = xr.Dataset(coords={'latitude': grafs_s0.lat.data, 'longitude': grafs_s0.lon.data,'time':flam_monthly.time})
flam_10km['flammability'] = (('time','latitude','longitude'), np.zeros((12,15,15)))
flam_10km.attrs = flam_monthly.attrs

In [ ]:
# for all time step 
for timestamp in flam_monthly.time:
    # Start by selecting the timestamp
    print(timestamp.data)
    
    # your code here
    
     

In [ ]:
# plot the time series
flam_10km.flammability.plot.imshow(col='time',col_wrap=6, cmap='OrRd',robust=True)

### 2.3 can you calculate the correlation between soil wetness and flammability for each pixel?